# **Importing Necessary Libraries**

In [1]:
import cv2
from google.colab.patches import cv2_imshow
import sys
import numpy as np

# **Detector**

In [2]:
def Detector (video, resize):
  '''
  Detects and skeletonize the humans.
  Also returns the modified video to 'output.avi'

  Parameters:
    'video': str
      path to the video
    'resize': tuple of integers (Width, Height)
      resizes each frame to specified values. 

  ''' 

  # Initializing the HOG person detector
  hog = cv2.HOGDescriptor()
  hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

  
  #for reading frames in video
  cap = cv2.VideoCapture(video)

  if (cap.isOpened()== False):
    sys.exit("Error opening video stream or file")

  #for storing video
  out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 25, resize)

  while(True):

    ret, frame = cap.read()

    if ret == True:
      
      #Resizing
      frame =cv2.resize(frame, resize)
      
    
      # detect humans
      (regions, _) = hog.detectMultiScale(frame, winStride=(4, 4), padding=(4, 4), scale=1.05, 	useMeanshiftGrouping = True)
      

      # Draw regions 
      for (x, y, w, h) in regions:
          cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
          
          crop = frame[y:y+h, x:x+w]

          #grayscale
          crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
          
          # detect corners with the goodFeaturesToTrack function.
          corners = cv2.goodFeaturesToTrack(crop, 27, 0.01, 10)
          corners = np.int0(corners)
          
          width, height = crop.shape
          
          #plot each corner and draw line from centre of rectangle to it
          for i in corners:
            a, b = i.ravel()
            
            #Removing outliers in the crops (rectangles)
            if a < (0.15* height) or a > (0.85*height):
              continue

            elif  b < (0.15* width) or b > (0.85*width):
              continue
            else:

              #adjusting for original image size
              a = a +x 
              b= b + y  

              cv2.circle(frame, (a, b), 2, (255,0,0), 2)
              cv2.line( frame, ((np.int(height/2)+x), (np.int(width/2)+y)), (a,b), (0,255,0), 1)

      #Write the frame into the file 'output.avi'
      out.write(frame)
        
    else:
      break 

  # release the video capture and video write objects
  cap.release()
  out.release()

# **Output**

In [3]:
Detector('/content/Input.mov', (640,360))